<a href="https://colab.research.google.com/github/umaabhyankar/AQI-Predictor/blob/main/feature_view_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U hopsworks --quiet

!pip install xgboost --q

In [2]:
import os
import datetime
import time
import json
import pickle
import joblib

import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/72854
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality',
    version = 1
)

In [42]:
query = air_quality_fg.select_all()

In [43]:
feature_view = fs.get_or_create_feature_view(
    name='air_quality_fv',
    version=1,
    query=query
)

In [46]:
td_version, td_job = feature_view.create_training_data(
    description = 'Air Quality dataset',
    data_format = 'csv',
    write_options = {'wait_for_job': True}
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/72854/jobs/named/air_quality_fv_1_create_fv_td_10072023081905/executions


FeatureStoreException: ignored

In [ ]:
X, _ = feature_view.get_training_data(
    training_dataset_version=td_version
)